# MAE  evaluation plots

In [ ]:
import numpy as np
import pandas as pd
import os

## Config path

In [ ]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'
RESULT_PATH = '/root/work/artwork_sequence/predicted_tours'
PLOT_PATH = '/root/work/artwork_sequence/plots'

In [ ]:
def get_predicted_tour_path(window_index):
    predicted_tour_base_path = os.path.join(RESULT_PATH,'config_'+str(window_index))

    predicted_tour_path = {
        'sequence_most_similar' : os.path.join(predicted_tour_base_path,'generated_sequence_based_previous_most_similar'),
        'sequence_rnn' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn'),
       # 'sequence_rnn_multivariate' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn_multivariate'),
        'sequence_rnn_code_embedding' : os.path.join(predicted_tour_base_path,'generated_sequence_rnn_code_embedding'),
        'sequence_rnn_artist_code_embedding' : os.path.join(predicted_tour_base_path, 'generated_sequence_rnn_artist_code_embedding')


    }

    return predicted_tour_path

In [ ]:
window_indexes = list(range(3,9))

window_config = {}

for index in window_indexes:
    window_config[index] = get_predicted_tour_path(index)
    

## Load data

In [ ]:
def load_mae_data(predicted_tour_path):
    #load data
    mae_dict = {}
    for key, value in predicted_tour_path.items():
        mae_dict[key] = pd.read_csv(os.path.join(value,'mae.csv'))
    
    #Add model name to the dataframe
    for key, df in mae_dict.items():
        df['model'] = key

    #Concat all dataframe in one to plot MAE
    df_mae = pd.concat(list(mae_dict.values()))
    df_mae = df_mae.reset_index(drop=False)

    return df_mae

In [ ]:
window_mae = {}

for index, path in window_config.items():
    window_mae[index] = load_mae_data(path)

## Plot MAE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Plot mae for each tour

In [ ]:
#Define nums of cols and rows in the plot
ncols = 2
nrows = len(window_indexes)//ncols

#Divide indixes in two set for plot
low_half_indexes = window_indexes[:len(window_indexes)//ncols]
high_half_indexes = window_indexes[len(window_indexes)//ncols:]

In [ ]:
list(window_mae.keys())

In [ ]:
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16,10))

fig.suptitle('MAE', fontsize=20)

index = window_indexes[0]

for i in range(ncols):
    for j in range(nrows):
        df_mae = window_mae[index]
        #Plot
        ax = sns.lineplot('index', 'mae', hue='model', data=df_mae, ax=axs[j,i])
        ax.set_title('Window size ' + str(index), fontdict={'fontsize': 14, 'fontweight': 'medium'} )
        ax.set_xlabel("Tour",fontsize=12)
        ax.set_ylabel("mae",fontsize=12)
        ax.set_ylim(bottom=0.2, top=0.65)

        index +=1
        
    

fig.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()

In [ ]:
fig.savefig(os.path.join(PLOT_PATH,"mae_tour.png"), dpi=100)

### Plot Global MAE

In [ ]:
def get_mae_global(model):
    
    df_mae_global = pd.DataFrame({ 'model' : [],
                         'index' : [],
                         'mae_global' : []})
    
    
    for index in window_indexes:
        df_mae = window_mae[index]
        mean_mae = df_mae[df_mae['model'] == model]['mae'].mean()
        
        #Save MAE
        df_mae_global = df_mae_global.append(
            {'model' : model,
             'index': index,
             'mae_global': mean_mae
            }, 
           ignore_index=True)

    
    return df_mae_global

In [ ]:
models = list(get_predicted_tour_path(3).keys())

df_mae_global_list = []

for m in models:
    df_mae_global = get_mae_global(m)
    df_mae_global_list.append(df_mae_global)

df_mae_global = pd.concat(df_mae_global_list)

In [ ]:
fig= plt.figure(figsize=(10,8))

ax = sns.lineplot(x="index", y='mae_global', hue='model', data=df_mae_global)

ax.set_xlabel("Window size",fontsize=12)
ax.set_ylabel("mae",fontsize=12)
ax.set_ylim(bottom=0.35, top=0.55)

In [ ]:
fig.savefig(os.path.join(PLOT_PATH,"mae_global.png"), dpi=100)

**MAE  Global for each model**

In [ ]:
df_mae_global[['model', 'mae_global']].groupby('model').mean()